## Inicializadores

In [2]:
%%capture
# pip install googletrans==4.0.0-rc1

In [3]:
import pandas as pd
from googletrans import Translator

In [4]:
df = pd.read_csv('../data/summaries_train.csv')
df.head()

,student_id,prompt_id,text,content,wording
0,000e8c3c7ddb,814d6b,The third wave was an experimentto see how peo...,0.205683,0.380538
1,0020ae56ffbf,ebad26,They would rub it up with soda to make the sme...,-0.548304,0.506755
2,004e978e639e,3b9047,"In Egypt, there were many occupations and soci...",3.128928,4.231226
3,005ab0199905,3b9047,The highest class was Pharaohs these people we...,-0.210614,-0.471415
4,0070c9e7af47,814d6b,The Third Wave developed rapidly because the ...,3.272894,3.219757


In [5]:
df_questions = pd.read_csv('../data/prompts_train.csv')
df_questions.head()

,prompt_id,prompt_question,prompt_title,prompt_text
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...
1,3b9047,"In complete sentences, summarize the structure...",Egyptian Social Structure,Egyptian society was structured like a pyramid...
2,814d6b,Summarize how the Third Wave developed over su...,The Third Wave,Background \r\nThe Third Wave experiment took ...
3,ebad26,Summarize the various ways the factory would u...,Excerpt from The Jungle,"With one member trimming beef in a cannery, an..."


### Traduzindo Os dataframes

In [12]:
# from googletrans import Translator
from translate import Translator

In [13]:
# Função para traduzir um texto para outro idioma
def translate_text(text, target_language):
    translator = Translator(to_lang=target_language)
    translation = translator.translate(text)
    return translation

In [15]:
df_questions_por = df_questions.copy()
target_language = 'pt'
df_questions_por['questao_proposta'] = df_questions['prompt_question'].apply(lambda x: translate_text(x, target_language))
df_questions_por['questao_titulo'] = df_questions['prompt_title'].apply(lambda x: translate_text(x, target_language))
# df_questions_por['questao_texto'] = df_questions['prompt_text'].apply(lambda x: translate_text(x, target_language))


In [ ]:
df_questions_por.to_csv('../data/df_questions_por.csv',index=False,sep=";")

In [ ]:
df_copy = df.copy()
target_language = 'pt'
df_copy['texto'] = df['text'].apply(lambda x: translate_text(x, target_language))


In [ ]:
df_copy.head()

In [ ]:
df_copy.to_csv('../data/sumaries_pt.csv',index=False,sep=";")

In [ ]:
# instalando o transformers
#pip install transformers
#pip install --upgrade transformers torch

### Carregando os modelos e tokenizers

In [6]:
from transformers import AutoTokenizer, AutoModel


In [7]:
#somente bert
bert_model = AutoModel.from_pretrained("bert-base-uncased")
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [8]:
#somente gpt
gpt_model = AutoModel.from_pretrained("gpt2")
gpt_tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [7]:
text = "Exemplo de texto para teste."

# Tokenizar o texto com o tokenizador do BERT
bert_inputs = bert_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# Executar o modelo BERT nos inputs
bert_outputs = bert_model(**bert_inputs)


In [ ]:
bert_outputs

## criando um modelo bert para avaliação gramatical

In [9]:
print(df_questions.columns)
print(df.columns)

Index(['prompt_id', 'prompt_question', 'prompt_title', 'prompt_text'], dtype='object')
Index(['student_id', 'prompt_id', 'text', 'content', 'wording'], dtype='object')


In [10]:
#juntando os DF
merged_df = df_questions.merge(df, on='prompt_id')
merged_df.head()

,prompt_id,prompt_question,prompt_title,prompt_text,student_id,text,content,wording
0,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00791789cc1f,1 element of an ideal tragedy is that it shoul...,-0.210614,-0.471415
1,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0086ef22de8f,The three elements of an ideal tragedy are: H...,-0.970237,-0.417058
2,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,0094589c7a22,Aristotle states that an ideal tragedy should ...,-0.387791,-0.584181
3,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00cd5736026a,One element of an Ideal tragedy is having a co...,0.088882,-0.594710
4,39c16e,Summarize at least 3 elements of an ideal trag...,On Tragedy,Chapter 13 \r\nAs the sequel to what has alrea...,00d98b8ff756,The 3 ideal of tragedy is how complex you need...,-0.687288,-0.460886


In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import pandas as pd

# Extraindo os dados relevantes do DataFrame merged_df
text_data = merged_df['text'].tolist()  # Textos das redações
content_data = merged_df['content'].tolist()  # Notas (conteúdo) das redações
wording_data = merged_df['wording'].tolist()  # Notas (linguagem) das redações
prompt_question_data = merged_df['prompt_question'].tolist()  # Perguntas associadas
prompt_text_data = merged_df['prompt_text'].tolist()  # Textos de prompt associados

# Carregar modelo BERT e tokenizador para o inglês
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 rótulos: content e wording
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preparar as entradas para o modelo
input_texts = [f"{text} {prompt_question} {prompt_text}" for text, prompt_question, prompt_text in zip(text_data, prompt_question_data, prompt_text_data)]

# Tokenizar e codificar os textos
encoded_texts = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converter notas de content e wording para tensores do PyTorch
notas_content = torch.tensor(content_data, dtype=torch.float32)
notas_wording = torch.tensor(wording_data, dtype=torch.float32)

# Concatenar as notas para criar um tensor de rótulos (target)
rotulos = torch.stack((notas_content, notas_wording), dim=1)

# Definir otimizador e treinamento
optimizer = AdamW(model.parameters(), lr=1e-5)

# Treinamento
model.train()
for epoch in range(5):
    optimizer.zero_grad()
    outputs = model(**encoded_texts, labels=rotulos)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    print(f"Época {epoch+1}, Perda: {loss.item()}")


In [12]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW


# Extraindo os dados relevantes
text_data = merged_df['text'].tolist()  # Textos das redações
content_data = merged_df['content'].tolist()  # Notas (conteúdo) das redações
wording_data = merged_df['wording'].tolist()  # Notas (linguagem) das redações
prompt_question_data = merged_df['prompt_question'].tolist()  # Perguntas associadas
prompt_text_data = merged_df['prompt_text'].tolist()  # Textos de prompt associados

# Carregar modelo BERT e tokenizador para o inglês
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)  # 2 rótulos: content e wording
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Preparar as entradas para o modelo
input_texts = [f"{text} {prompt_question} {prompt_text}" for text, prompt_question, prompt_text in zip(text_data, prompt_question_data, prompt_text_data)]

# Tokenizar e codificar os textos
encoded_texts = tokenizer(input_texts, padding=True, truncation=True, return_tensors='pt', max_length=128)

# Converter notas de content e wording para tensores do PyTorch
notas_content = torch.tensor(content_data, dtype=torch.float32)
notas_wording = torch.tensor(wording_data, dtype=torch.float32)

# Concatenar as notas para criar um tensor de rótulos (target)
rotulos = torch.stack((notas_content, notas_wording), dim=1)

# Definir otimizador e treinamento
optimizer = AdamW(model.parameters(), lr=1e-5)

batch_size = 8  # Tamanho do lote
gradient_accumulation_steps = 2  # Acumulação de gradientes a cada 2 mini-lotes

model.train()
for epoch in range(5):
    optimizer.zero_grad()
    for batch_start in range(0, len(encoded_texts['input_ids']), batch_size):
        batch_end = batch_start + batch_size
        batch_inputs = {k: v[batch_start:batch_end] for k, v in encoded_texts.items()}
        batch_labels = rotulos[batch_start:batch_end]
        outputs = model(**batch_inputs, labels=batch_labels)
        loss = outputs.loss
        loss.backward()
        if (batch_start // batch_size) % gradient_accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
    print(f"Época {epoch+1}, Perda: {loss.item()}")


Época 3, Perda: -3.9181365966796875
Época 4, Perda: -4.733118534088135
Época 5, Perda: -5.846802711486816


## Salvando o Modelo

In [15]:
caminho_salvar = "../data"

# Salvando o modelo
model.save_pretrained(caminho_salvar)
tokenizer.save_pretrained(caminho_salvar)


('../data\\tokenizer_config.json',
 '../data\\special_tokens_map.json',
 '../data\\vocab.txt',
 '../data\\added_tokens.json')

In [30]:
text_sample = merged_df[:1]['text'].values[0]
print(text_sample)

1 element of an ideal tragedy is that it should be arranged on a complex plan.  Another element of an ideal tragedy is that it should only have one main issue. The last element of an ideal tragedy is that it should have a double thread plot and an opposite catastrophe for both good and bad.


In [31]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

# Caminho onde o modelo foi salvo
caminho_modelo = caminho_salvar

# Carregar modelo e tokenizador
model = BertForSequenceClassification.from_pretrained(caminho_modelo)
tokenizer = BertTokenizer.from_pretrained(caminho_modelo)

# Texto para avaliar
texto_a_avaliar = text_sample

# Tokenizar e codificar o texto
input_text = tokenizer(texto_a_avaliar, padding=True, truncation=True, return_tensors='pt', max_length=128)

# Fazer a previsão
model.eval()
with torch.no_grad():
    outputs = model(**input_text)
    notas_content, notas_wording = outputs.logits[0]

print(f"Nota de Conteúdo: {notas_content.item()}")
print(f"Nota de Linguagem: {notas_wording.item()}")


Nota de Conteúdo: 25.637664794921875
Nota de Linguagem: 23.27189064025879


In [32]:
merged_df[:1][['content','wording']]

,content,wording
0,-0.210614,-0.471415
